In [1]:
import os
import os.path
import cv2
import glob
import h5py
import tqdm
import argparse
import logging
from PIL import Image 
import skimage
import skimage.io

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
sns.set_theme()
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms

import data, utils, models

In [2]:
def load_model(PATH, parallel=True, pretrained=True, old=True, load_opt=False):
    state_dict = torch.load(PATH, map_location="cpu")
    args = argparse.Namespace(**{**vars(state_dict["args"])})
    if old:
        vars(args)['blind_noise'] = False
    
    model = models.build_model(args)
    # model.load_state_dict(state_dict["model"][0])
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    if load_opt:
        for o, state in zip([optimizer], state_dict["optimizer"]):
            o.load_state_dict(state)
    
    if pretrained:
        state_dict = torch.load(PATH)["model"][0]
        own_state = model.state_dict()
        # print(own_state)

        for name, param in state_dict.items():
            if parallel:
                name = name[7:]
            if name not in own_state:
                print("here", name)
                continue
            if isinstance(param, nn.Parameter):
                # backwards compatibility for serialized parameters
                param = param.data
            own_state[name].copy_(param)
        
    return model, optimizer, args

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# PATH = "/scratch/ds6516/video_denoising/experiments/blind-video-net-4/raw-video-BF-0-100-Mar-14-05:36:55/checkpoints/checkpoint_best.pt"
PATH = "/scratch/ds6516/video_denoising/experiments/blind-video-net-4/raw-video-BF-0-100-Mar-15-01:56:48/checkpoints/checkpoint_best.pt"
# PATH = "/scratch/ds6516/video_denoising/experiments/blind-video-net-4/raw-iso25600-BF-0-100-Mar-15-07:20:36/checkpoints/checkpoint_best.pt"

model, optimizer, args = load_model(PATH)
model.to(device)
print(model)

BlindVideoNet(
  (rotate): rotate()
  (denoiser_1): Blind_UNet(
    (enc1): ENC_Conv(
      (conv1): Conv(
        (shift_down): ZeroPad2d(padding=(0, 0, 1, 0), value=0.0)
        (crop): crop()
        (replicate): ReplicationPad2d((1, 1, 1, 1))
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (relu): LeakyReLU(negative_slope=0.1, inplace=True)
      )
      (conv2): Conv(
        (shift_down): ZeroPad2d(padding=(0, 0, 1, 0), value=0.0)
        (crop): crop()
        (replicate): ReplicationPad2d((1, 1, 1, 1))
        (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (relu): LeakyReLU(negative_slope=0.1, inplace=True)
      )
      (conv3): Conv(
        (shift_down): ZeroPad2d(padding=(0, 0, 1, 0), value=0.0)
        (crop): crop()
        (replicate): ReplicationPad2d((1, 1, 1, 1))
        (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (relu): LeakyReLU(negative_slope=0.1, inplace=True)
  

In [4]:
class RawVideo(torch.utils.data.Dataset):
    def __init__(self, data_path, datatype="train", patch_size=None, stride=64, n_frames=5, aug=0,
                 scenes=[7, 8, 9, 10, 11],
                 isos = [1600, 3200, 6400, 12800, 25600]):
        super().__init__()
        self.data_path = data_path
        self.datatype = datatype
        self.size = patch_size
        self.stride = stride
        self.n_frames = n_frames
        self.aug = aug
        
        self.noisy_path = os.path.join(self.data_path, "indoor_raw_noisy")
        self.gt_path = os.path.join(self.data_path, "indoor_raw_gt")
        self.scenes = scenes
        self.isos = isos
        if self.datatype == "train":
            self.nr = 9 # noise_realisations
        elif self.datatype == "val":
            self.nr = 1 # only the 9th noise realisation used for heldout
        elif self.datatype == "test":
            self.nr = 10
        self.fpv = self.bound = 7 # frames_per_video
        
        self.len = self.fpv * self.nr * len(self.isos) * len(self.scenes)
        
        self.transform = transforms.Compose([transforms.ToTensor()])
        self.reverse = transforms.Compose([transforms.ToPILImage()])

        Img = skimage.io.imread(os.path.join(self.noisy_path, f"scene{self.scenes[0]}", 
                                             f"ISO{self.isos[0]}", "frame1_noisy0.tiff"))
        H, W = Img.shape
        
        if self.size is not None:
            self.n_H = (int((H-self.size)/self.stride)+1)
            self.n_W = (int((W-self.size)/self.stride)+1)
            self.n_patches = self.n_H * self.n_W
            self.len *= self.n_patches

        self.hflip = transforms.Compose([transforms.RandomHorizontalFlip(p=1)])
        self.vflip = transforms.Compose([transforms.RandomVerticalFlip(p=1)])

        if aug >= 1: # Horizonatal and Vertical Flips
            self.len *= 4
        if aug >= 2: # Reverse the Video
            self.len *= 2
        
    def __len__(self):
        return self.len

    def __getitem__(self, index):
        hop = 1
        reverse = 0
        flip = 0
        if self.aug >= 2: # Reverse the Video
            reverse = index % 2
            index = index // 2
        if self.aug >= 1: # Horizonatal and Vertical Flips
            flip = index % 4
            index = index // 4

        if self.size is not None:
            patch = index % self.n_patches
            index = index // self.n_patches
            
        scene = index % len(self.scenes)
        index = index // len(self.scenes)
        
        iso = index % len(self.isos)
        index = index // len(self.isos)
        
        if self.datatype == "val":
            nr = 9
        else:
            nr = index % self.nr
            index = index // self.nr
        
#         print(scene, iso, nr, index)
        
        ends = 0
        x = ((self.n_frames-1) // 2)*hop
        if index < x:
            ends = x - index
        elif self.bound-1-index < x:
            ends = -(x-(self.bound-1-index))

        Img = skimage.io.imread(os.path.join(self.gt_path, 
                                             f"scene{self.scenes[scene]}",
                                             f"ISO{self.isos[iso]}", 
                                             f"frame{index+1}_clean_and_slightly_denoised.tiff"))
        H, W = Img.shape
        Img = Img.reshape(H, W, 1)
        noisy_Img = skimage.io.imread(os.path.join(self.noisy_path, 
                                             f"scene{self.scenes[scene]}",
                                             f"ISO{self.isos[iso]}", 
                                             f"frame{index+1}_noisy{nr}.tiff"))
        noisy_Img = noisy_Img.reshape(H, W, 1)

        for i in range(hop, x+1, hop):
            end = max(0, ends)
            off = max(0,i-x+end)
#             print(index-i+off)
            # img = Image.open(self.files[index-i+off])
            img = skimage.io.imread(os.path.join(self.gt_path, 
                                             f"scene{self.scenes[scene]}",
                                             f"ISO{self.isos[iso]}", 
                                             f"frame{index-i+off+1}_clean_and_slightly_denoised.tiff"))
            img = img.reshape(H, W, 1)
            # noisy_img = np.load(self.noisy_files[index-i+off])
            noisy_img = skimage.io.imread(os.path.join(self.noisy_path, 
                                             f"scene{self.scenes[scene]}",
                                             f"ISO{self.isos[iso]}", 
                                             f"frame{index-i+off+1}_noisy{nr}.tiff"))
            noisy_img = noisy_img.reshape(H, W, 1)
            
            if reverse == 0:
                Img = np.concatenate((img, Img), axis=2)
                noisy_Img = np.concatenate((noisy_img, noisy_Img), axis=2)
            else:
                Img = np.concatenate((Img, img), axis=2)
                noisy_Img = np.concatenate((noisy_Img, noisy_img), axis=2)

        for i in range(hop, x+1, hop):
            end = -min(0,ends)
            off = max(0,i-x+end)
#             print(index+i-off)
            # img = Image.open(self.files[index+i-off])
            img = skimage.io.imread(os.path.join(self.gt_path, 
                                             f"scene{self.scenes[scene]}",
                                             f"ISO{self.isos[iso]}", 
                                             f"frame{index+i-off+1}_clean_and_slightly_denoised.tiff"))
            img = img.reshape(H, W, 1)
            # noisy_img = np.load(self.noisy_files[index+i-off])
            noisy_img = skimage.io.imread(os.path.join(self.noisy_path, 
                                             f"scene{self.scenes[scene]}",
                                             f"ISO{self.isos[iso]}", 
                                             f"frame{index+i-off+1}_noisy{nr}.tiff"))
            noisy_img = noisy_img.reshape(H, W, 1)
            
            if reverse == 0:
                Img = np.concatenate((Img, img), axis=2)
                noisy_Img = np.concatenate((noisy_Img, noisy_img), axis=2)
            else:
                Img = np.concatenate((img, Img), axis=2)
                noisy_Img = np.concatenate((noisy_img, noisy_Img), axis=2)

        if self.size is not None:
            nh = (patch // self.n_W)*self.stride
            nw = (patch % self.n_W)*self.stride
            Img = Img[nh:(nh+self.size), nw:(nw+self.size), :]
            noisy_Img = noisy_Img[nh:(nh+self.size), nw:(nw+self.size), :]

        if flip == 1:
            Img = np.flip(Img, 1)
            noisy_Img = np.flip(noisy_Img, 1)
        elif flip == 2:
            Img = np.flip(Img, 0)
            noisy_Img = np.flip(noisy_Img, 0)
        elif flip == 3:
            Img = np.flip(Img, (1,0))
            noisy_Img = np.flip(noisy_Img, (1,0))
        
        Img = Img.astype(np.float32)
        noisy_Img = noisy_Img.astype(np.float32)
#         print(Img.max(), Img.min(), noisy_Img.max(), noisy_Img.min())
        Img = (Img-240.)/(2**12-1-240)
        noisy_Img = (noisy_Img-240.)/(2**12-1-240)
#         print(Img.max(), Img.min(), noisy_Img.max(), noisy_Img.min())

        # return self.transform(np.array(Img)).type(torch.FloatTensor)
        # return self.transform(np.array(Img)).type(torch.FloatTensor), torch.from_numpy(noisy_Img.copy())
        return self.transform(np.array(Img)).type(torch.FloatTensor), self.transform(np.array(noisy_Img)).type(torch.FloatTensor)        

In [16]:
path = "/scratch/ds6516/video_denoising/datasets/RawVideo"

# dataset = RawVideo(path, datatype="train", patch_size=128, stride=64, aug=0)
# dataset = RawVideo(path, datatype="val", patch_size=1080, stride=1920-1080)
dataset = RawVideo(path, datatype="val", patch_size=1080, stride=1920-1080, isos=[25600])
len(dataset)

70

In [18]:
# Test
model.eval()

valid_loader = torch.utils.data.DataLoader(dataset, batch_size=1, num_workers=1, shuffle=False)

cpf = 1
mid = 2

valid_meters = {name: utils.AverageMeter() for name in (["valid_psnr", "valid_ssim"])}

model.eval()
for meter in valid_meters.values():
    meter.reset()

valid_bar = utils.ProgressBar(valid_loader)
running_valid_psnr = 0.0
plist = []
slist = []
for sample_id, (sample, noisy_inputs) in enumerate(valid_bar):
    with torch.no_grad():
        sample = sample.to(device)
        noisy_inputs = noisy_inputs.to(device)
        
        outputs, est_sigma = model(noisy_inputs)
        
        valid_psnr = utils.psnr(sample[:, (mid*cpf):((mid+1)*cpf), :, :], outputs, normalized=False, raw=True)
        valid_ssim = utils.ssim(sample[:, (mid*cpf):((mid+1)*cpf), :, :], outputs, normalized=False, raw=True)
        plist.append(valid_psnr)
        slist.append(valid_ssim)
        running_valid_psnr += valid_psnr
        valid_meters["valid_psnr"].update(valid_psnr.item())
        valid_meters["valid_ssim"].update(valid_ssim.item())
        
        valid_bar.log(dict(**valid_meters, lr=optimizer.param_groups[0]["lr"]), verbose=True)
        
running_valid_psnr /= (sample_id+1)
print("EVAL: "+valid_bar.print(dict(**valid_meters, lr=optimizer.param_groups[0]["lr"])))
# print(plist)
# print(slist)

EVAL: valid_psnr 42.113 | valid_ssim 0.995 | lr 1.0e-04


- [raw-random-eval-full] (43.97) EVAL: valid_psnr 42.952 | valid_ssim 0.996 | lr 1.0e-04
- [raw-random-eval-full-ISO25600] (41.17) EVAL: valid_psnr 40.939 | valid_ssim 0.994 | lr 1.0e-04
- [raw-random-eval-full-ISO1600] (47.74) EVAL: valid_psnr 45.881 | valid_ssim 0.998 | lr 1.0e-04
- 
- [raw-full] EVAL: valid_psnr 44.075 | valid_ssim 0.997 | lr 1.0e-04
- [raw-full-ISO1600] EVAL: valid_psnr 47.927 | valid_ssim 0.998 | lr 1.0e-04 (latest)
- [raw-full-ISO3200] EVAL: valid_psnr 45.844 | valid_ssim 0.998 | lr 1.0e-04
- [raw-full-ISO6400] EVAL: valid_psnr 44.092 | valid_ssim 0.997 | lr 1.0e-04
- [raw-full-ISO12800] EVAL: valid_psnr 41.512 | valid_ssim 0.996 | lr 1.0e-04
- [raw-full-ISO25600] EVAL: valid_psnr 41.515 | valid_ssim 0.994 | lr 1.0e-04
-
- [raw-ISO25600] EVAL: valid_psnr 41.375 | valid_ssim 0.994 | lr 1.0e-04
-
- EVAL: valid_psnr 44.694 | valid_ssim 0.997 | lr 1.0e-04
- [1600] EVAL: valid_psnr 48.036 | valid_ssim 0.998 | lr 1.0e-04
- [3200] EVAL: valid_psnr 46.424 | valid_ssim 0.998 | lr 1.0e-04
- [6400] EVAL: valid_psnr 44.705 | valid_ssim 0.997 | lr 1.0e-04
- [12800] EVAL: valid_psnr 42.189 | valid_ssim 0.997 | lr 1.0e-04
- [25600] EVAL: valid_psnr 42.113 | valid_ssim 0.995 | lr 1.0e-04

In [ ]:
save = 0
x = 120; y = 120; w = 320; h = 320
bright = 0.6

with torch.no_grad():
    model.eval()
    for i in [5,7,9]:
        inputs = dataset[i][1].unsqueeze(0).to(device)
        outputs, _ = model(inputs)
        
        fig, ax = plt.subplots(figsize=(3,3))
        
        img = inputs[0,2,:,:].cpu().detach().numpy()
#         ax.imshow(img, cmap="gray", vmin=0, vmax=1)
        img = np.uint16(img * (2**12-1-240) + 240)
        img = cv2.cvtColor(img, cv2.COLOR_BayerGR2RGB)
        img = (img-240.)/(2**12-1-240)
        ax.imshow(img[y:y+h, x:x+w, :]/bright)
        ax.axis("off")

        plt.tight_layout()
        
        if save==1:
            plt.savefig(f"./plots/raw_videos/scene_11_{i}_noisy.pdf", bbox_inches="tight", pad_inches=0)
            plt.close()
        
        fig, ax = plt.subplots(figsize=(3,3))
        
        img = outputs[0,0,:,:].cpu().detach().numpy()
#         ax.imshow(img, cmap="gray", vmin=0, vmax=1)
        img = np.uint16(img * (2**12-1-240) + 240)
        img = cv2.cvtColor(img, cv2.COLOR_BayerGR2RGB)
        img = (img-240.)/(2**12-1-240)
        ax.imshow(img[y:y+h, x:x+w, :]/bright)
        ax.axis("off")

        plt.tight_layout()
        
        if save==1:
            plt.savefig(f"./plots/raw_videos/scene_11_{i}_denoised.pdf", bbox_inches="tight", pad_inches=0)
            plt.close()
        
        fig, ax = plt.subplots(figsize=(3,3))
        
        img = dataset[i][0][2,:,:].cpu().detach().numpy()
#         ax.imshow(img, cmap="gray", vmin=0, vmax=1)
        img = np.uint16(img * (2**12-1-240) + 240)
        img = cv2.cvtColor(img, cv2.COLOR_BayerGR2RGB)
        img = (img-240.)/(2**12-1-240)
        ax.imshow(img[y:y+h, x:x+w, :]/bright)
        ax.axis("off")

        plt.tight_layout()
        
        if save==1:
            plt.savefig(f"./plots/raw_videos/scene_11_{i}_clean.pdf", bbox_inches="tight", pad_inches=0)
            plt.close()

In [ ]:
img.max()

In [ ]:
save = 0

# num = 9 
# aux = 11
# x = 100; y = 50; w = 320; h = 320; p = w // 2; q = h // 2;

num = 9 
aux = 10
x = 32; y = 200; w = 256; h = 256; p = w // 2; q = h // 2;

bright = 0.6

model.train()

inputs = dataset[num][1].unsqueeze(0)[:,:,y:y+h, x:x+w].to(device).requires_grad_(True)

outputs, _ = model(inputs)

loss = 100*outputs[0,0,q,p]

model.zero_grad()

loss.backward()

grads = inputs.grad.cpu().detach().numpy()

In [ ]:
limit = 0.3*np.max(np.abs(grads))

fig, ax = plt.subplots(figsize=(3,3))

# ax.imshow(inputs[0,2,:,:].cpu().detach().numpy(), cmap="gray", vmin=0, vmax=1)
img = inputs[0,2,:,:].cpu().detach().numpy()
img = np.uint16(img * (2**12-1-240) + 240)
img = cv2.cvtColor(img, cv2.COLOR_BayerGR2RGB)
img = (img-240.)/(2**12-1-240)
ax.imshow(img/bright)
ax.axis("off")

# DO NOT TOUCH
ax.set_axis_off()
plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
plt.margins(0,0)
ax.xaxis.set_major_locator(plt.NullLocator())
ax.yaxis.set_major_locator(plt.NullLocator())
##############

plt.tight_layout()

if save == 1:
    plt.savefig(f"./plots/raw_videos/input_{aux}_{num}.pdf", bbox_inches="tight", pad_inches=0)
    plt.close()

fig, ax = plt.subplots(figsize=(3,3))

# ax.imshow(outputs[0,0,:,:].cpu().detach().numpy(), cmap="gray", vmin=0, vmax=1)
img = outputs[0,0,:,:].cpu().detach().numpy()
img = np.uint16(img * (2**12-1-240) + 240)
img = cv2.cvtColor(img, cv2.COLOR_BayerGR2RGB)
img = (img-240.)/(2**12-1-240)
ax.imshow(img/bright)
# rect = patches.Rectangle((p-1,q-1),2,2,linewidth=1,edgecolor='r',facecolor='r')
# ax.add_patch(rect)
# circle = patches.Circle((p,q), radius=2, color='r')
# ax.add_patch(circle)
ax.hlines(h/2, xmin=0, xmax=w, linestyles='dashed', color='r')
ax.vlines(w/2, ymin=0, ymax=h, linestyles='dashed', color='r')
ax.axis("off")

# DO NOT TOUCH
ax.set_axis_off()
plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
plt.margins(0,0)
ax.xaxis.set_major_locator(plt.NullLocator())
ax.yaxis.set_major_locator(plt.NullLocator())
##############

plt.tight_layout()

if save == 1:
    plt.savefig(f"./plots/raw_videos/output_{aux}_{num}.pdf", bbox_inches='tight', pad_inches=0)
    plt.close()

for i in range(5):
    fig, ax = plt.subplots(figsize=(3,3))
    
    ax.imshow(grads[0,i,:,:], cmap="seismic", vmin=-limit, vmax=limit)
#     rect = patches.Rectangle((p-1,q-1),2,2,linewidth=1,edgecolor='limegreen',facecolor='none')
#     ax.add_patch(rect)
#     if i==2:
#         poly = patches.RegularPolygon((p-1,q-1), numVertices=3, radius=3, color='limegreen')
#         ax.add_patch(poly)
#     else:
#         circle = patches.Circle((p-1,q-1), radius=2, color='limegreen')
#         ax.add_patch(circle)

    ax.hlines(h/2, xmin=0, xmax=w, linestyles='dashed', color='gray')
    ax.vlines(w/2, ymin=0, ymax=h, linestyles='dashed', color='gray')
    
    rect = patches.Rectangle((0,0),w-1,h-1,linewidth=2,edgecolor="midnightblue" ,facecolor='none')
    ax.add_patch(rect)
    ax.axis("off")
    
    # DO NOT TOUCH
    ax.set_axis_off()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
                hspace = 0, wspace = 0)
    plt.margins(0,0)
    ax.xaxis.set_major_locator(plt.NullLocator())
    ax.yaxis.set_major_locator(plt.NullLocator())
    ##############
    
    plt.tight_layout()
    
    if save == 1:
        plt.savefig(f"./plots/raw_videos/grads_{aux}_{num}_{i}.pdf", bbox_inches='tight', pad_inches=0)
        plt.close()
    
plt.figure(figsize = (3,3))
img = plt.imshow(grads[0,2,:,:], cmap="seismic", vmin=-limit, vmax=limit)
plt.xticks([])
plt.yticks([])
plt.tight_layout()

fig, ax = plt.subplots(figsize=(3,3))
cbar = plt.colorbar(img, ax=ax)
ax.remove()
plt.tight_layout()

if save == 1:
    plt.savefig(f"./plots/raw_videos/cbar_{aux}_{num}.pdf", bbox_inches='tight', pad_inches=0)
    plt.close()

In [ ]:
np.sum(grads[0,2,:,:]), dataset.data_path

In [14]:
video = "scene_10"
bright = 0.6

model.eval()

frames = []
os.makedirs(os.path.join(dataset.data_path, "denoised"), exist_ok=True)
save_dir = os.path.join(dataset.data_path, "denoised")
C, H, W = dataset[0][1].shape
x = 0; y = 540; w = 540; h = 540

with torch.no_grad():
    for i in [1,3,5,7,9,11,13]:
        noisy_inputs = dataset[i][1].unsqueeze(0)
        noisy_inputs = noisy_inputs[:,:,y:y+h,x:x+w].to(device)
        
        N, C, H, W = noisy_inputs.shape
        noisy_frame = noisy_inputs[:, 2:3, :, :]
        
        output, _ = model(noisy_inputs)
        
        img = np.array(output[0,0,:,:].cpu().detach())
        img = np.uint16(img * (2**12-1-240) + 240)
        img = cv2.cvtColor(img, cv2.COLOR_BayerGR2BGR)
        img = (img-240.)/(2**12-1-240)
        img /= bright
        img *= 255
        img = np.uint8(np.clip(img, 0, 255))
        
        noisy_img = np.array(noisy_frame[0,0,:,:].cpu().detach())
        noisy_img = np.uint16(noisy_img * (2**12-1-240) + 240)
        noisy_img = cv2.cvtColor(noisy_img, cv2.COLOR_BayerGR2BGR)
        noisy_img = (noisy_img-240.)/(2**12-1-240)
        noisy_img /= bright
        noisy_img *= 255
        noisy_img = np.uint8(np.clip(noisy_img, 0, 255))
        
        frame = np.append(noisy_img, img, axis=1)
        cv2.line(frame, (W,0), (W,H), (0,0,0), 1)
        
        cv2.imwrite(os.path.join(save_dir, "%05d.jpg" % (i)), frame)
        
        frames.append(frame)

In [15]:
fps = 12

video_path = os.path.join(save_dir, f"{video}.mp4")
height, width, layers = frames[0].shape
out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'MJPG'), fps, (width, height))
for i in range(len(frames)):
    out.write(frames[i])
out.release()
cv2.destroyAllWindows()

In [16]:
from moviepy.editor import *

video_path = os.path.join(save_dir, f"{video}.mp4")
gif_path = os.path.join(save_dir, f"{video}.gif")

clip = (VideoFileClip(video_path).resize(0.5))
clip.write_gif(gif_path)

t:  57%|█████▋    | 4/7 [00:00<00:00, 26.43it/s, now=None]

MoviePy - Building file /scratch/ds6516/video_denoising/datasets/RawVideo/denoised/scene_10.gif with imageio.


In [ ]:
from skimage.measure import compare_psnr,compare_ssim

utils.psnr(dataset[0][0].unsqueeze(0)[:,2:3,:,:], dataset[0][1].unsqueeze(0)[:,2:3,:,:], raw=False)

# print(compare_psnr(dataset[0][0][2].numpy(), dataset[0][1][2].numpy(), data_range=1.0))
# print(compare_ssim(dataset[0][0][2].numpy(), dataset[0][1][2].numpy(), data_range=1.0))

In [ ]:
with torch.no_grad():
    inputs = dataset[0][1].unsqueeze(0).to(device)[:,:,:1080,:1080]
    outputs, _ = model(inputs)

In [ ]:
clean = np.zeros((5,3,100,100))
noisy = np.zeros((5,3,100,100))
noisy.transpose(0,2,3,1).shape
# for c, n in zip(clean, noisy):
#     print(c[0].shape, n[0].shape)
#     break

In [ ]:
path = os.path.join(dataset.gt_path, 
             f"scene{dataset.scenes[1]}",
             f"ISO{dataset.isos[1]}", 
             f"frame{1}_clean_and_slightly_denoised.tiff")

img = skimage.io.imread(path)
img.max(), img.min(), path

In [ ]:
# test_gt = cv2.imread('../../my_raw_video_recorded_from_current_HuaweiCamera/indoor_raw_test_gt_slightly_denoised/scene{}/ISO{}/frame{}_clean_and_slightly_denoised.tiff'.format(scene_id, iso, i),-1).astype(np.float32)
# test_gt = (test_gt-240)/(2**12-1-240)

# test_raw_psnr = compare_psnr(test_gt,(np.uint16(test_result*(2**12-1-240)+240).astype(np.float32)-240)/(2**12-1-240), data_range=1.0)
# test_raw_ssim = compute_ssim_for_packed_raw(test_gt, (np.uint16(test_result*(2**12-1-240)+240).astype(np.float32)-240)/(2**12-1-240))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
# import cv2

img1 = Image.open("./datasets/RawVideo/indoor_raw_noisy/scene1/ISO25600/frame1_noisy0.tiff")
img2 = Image.open("./datasets/RawVideo/indoor_raw_gt/scene1/ISO25600/frame1_clean_and_slightly_denoised.tiff")
# img = cv2.imread("./datasets/RawVideo/indoor_raw_noisy/scene1/ISO25600/frame1_noisy0.tiff")
# img = cv2.cvtColor(img, cv2.COLOR_BayerGB2RGB)
img1 = np.array(img1)
img2 = np.array(img2)
img1 = (img1 - img1.min())
img1 = img1/img1.max()
img2 = (img2 - img2.min())
img2 = img2/img2.max()
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(20,20))
axs[0].imshow(img1, cmap="gray")
axs[0].axis("off")
axs[1].imshow(img2, cmap="gray")
axs[1].axis("off")
# print(img.mean(), img.max(), img.min(), img.shape)

In [ ]:
import os
import os.path
# import cv2
import glob

# PATH = "/scratch/ds6516/video_denoising/datasets/Derfs"
PATH = "/scratch/ds6516/video_denoising/datasets/GoPro"

# videos = ["park_joy", "sunflower", "touchdown", "tractor"]
videos = ["hypersmooth", "motorbike", "rafting", "snowboard"]

for name in videos:
    video = cv2.VideoCapture(os.path.join(PATH, name, name+".avi"))
    count = 1
    while(True):
        ret, frame = video.read()
        if ret:
            cv2.imwrite(os.path.join(PATH, name, "%05d.png" % (count)), frame)
            count += 1
        else:
            break
    video.release()
    cv2.destroyAllWindows()